In [ ]:
# res = compare_experiments_barplot(
#     experiment_paths=[experiment_output_dir],
#     title="TARS eval.",
# )


## WANDB dev

In [ ]:
from pathlib import Path

import pandas as pd
import yaml

from data_util import create_multi_label_train_test_splits

CONFIG = yaml.safe_load(
    Path(
        "/Users/samhardyhey/Desktop/blog/blog-multi-label/train/training_config.yaml"
    ).read_bytes()
)

# 1.1 create splits
df = pd.read_csv(CONFIG["dataset"])
train, test = create_multi_label_train_test_splits(
    df, label_col=CONFIG["label_col"], test_size=CONFIG["test_size"]
)
test, dev = create_multi_label_train_test_splits(
    test, label_col=CONFIG["label_col"], test_size=CONFIG["test_size"]
)

# # 1.2 log splits
# with wandb.init(
#     project=CONFIG["wandb_project"],
#     name="reddit_aus_finance",
#     group=CONFIG["wandb_group"],
#     entity="cool_stonebreaker",
# ) as run:
#     log_dataframe(run, train, "train_split", "Train split")
#     log_dataframe(run, dev, "dev_split", "Dev split")
#     log_dataframe(run, test, "test_split", "Test split")


## Dictionary classifier

In [ ]:
from model_util import fit_and_log_dictionary_classifier, fit_and_log_linear_svc

for model in CONFIG["models"]:
    model["model"]
    # if model['name'] == 'dictionary_classifier':
    #     fit_and_log_dictionary_classifier(train, dev, test, model)

    # elif model['name'] == 'sklearn_linear_svc':
    #     fit_and_log_linear_svc(train, dev, test, model)

    # else:
    #     print(f"Unsupported model: {model['name']} found")


In [ ]:
# mock multiple model runs for a single group
    model_config = CONFIG["models"][0]

    dc = DictionaryClassifier(
        classifier_type=model_config["classifier_type"],
        label_dictionary=model_config["label_dictionary"],
    )

    # get train/test performance > log as model artefacts?
    dev_pred = dev.assign(
        pred=lambda x: x[CONFIG["text_col"]].apply(dc.predict_single))
    test_pred = test.assign(
        pred=lambda x: x[CONFIG["text_col"]].apply(dc.predict_single))

    label_names = label_dictionary_to_label_mat(
        test_pred.pred).columns.tolist()
    class_report = (pd.DataFrame(classification_report(
        label_dictionary_to_label_mat(test[CONFIG["label_col"]]),
        label_dictionary_to_label_mat(test_pred.pred),
        target_names=label_names,
        output_dict=True,
    ))
        .T
        .reset_index()
        .rename(mapper={"index": "label"}, axis="columns", inplace=False)
    )

    slim_class_report = (
        class_report
        .query('label in @label_names')
        .pipe(lambda x: x[["label", "f1-score", "support"]])
        .set_index("label")
        .to_dict(orient="index")
    )

    # seperate models as seperate runs
    with wandb.init(
        project=CONFIG["wandb_project"],
        name=model_config['model'],
        group=CONFIG["wandb_group"],
        entity="cool_stonebreaker",
    ) as run:
        wandb.config.model = model_config['model']
        wandb.config.group = CONFIG["wandb_group"]

        # log dev/pred preds
        log_dataframe(run, dev_pred, "dev_preds", "Dev predictions")
        log_dataframe(run, test_pred, "test_preds", "Test predictions")

        run.log(slim_class_report)
        run.summary["test_f1"] = class_report.query(
            'label == "weighted avg"')['f1-score'].iloc[0]
        run.summary["test_support"] = class_report.query(
            'label == "weighted avg"')['support'].iloc[0]

    # seperate models as seperate runs
    with wandb.init(
        project=CONFIG["wandb_project"],
        name="flair_tars",
        group=CONFIG["wandb_group"],
        entity="cool_stonebreaker",
    ) as run:
        wandb.config.model = "flair_tars"
        wandb.config.group = CONFIG["wandb_group"]

        # log dev/pred preds
        log_dataframe(run, dev_pred, "dev_preds", "Dev predictions")
        log_dataframe(run, test_pred, "test_preds", "Test predictions")

        run.log(slim_class_report)
        run.summary["test_f1"] = class_report.query(
            'label == "weighted avg"')['f1-score'].iloc[0]
        run.summary["test_support"] = class_report.query(
            'label == "weighted avg"')['support'].iloc[0]

## WANDB misc

In [ ]:
# clear out for dev purposes
import wandb

api = wandb.Api()

for run in api.runs(path="cool_stonebreaker/tyre_kick"):
    run = api.run(f"cool_stonebreaker/tyre_kick/{run.id}")
    run.delete()


In [ ]:
!pip install plotly

In [ ]:
import matplotlib.pyplot as plt

import wandb

fibonacci = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
plt.plot(fibonacci)
plt.ylabel("some interesting numbers")

# Initialize run
with wandb.init(
    project=CONFIG["wandb_project"],
    name="flair_tars",
    group=CONFIG["wandb_group"],
    entity="cool_stonebreaker",
) as run:

    # Log plot object
    wandb.log({"plot": plt})
